# Python, Data, 2025

---

* [Skutečné hodnoty](#Skutečné-datové-sety),
    - [chybějící údaje](#Kolik-mi-chybí-údajů),
    - [encoding](#Encoding),
    - [nekonzistentní data](#Nekonzistentní-data).

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JJu86kKRph1LHt5M7agODQHaHa%26pid%3DApi&f=1&ipt=5cfb9e9f7b451b3a4f7a3ebdc4eb99177d477f20f8145cd818cc4966143f8b2b&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

## Skutečné datové sety

---

Čištění dat je proces, který patří k samotné práci s daty.

Spolu s některými souvisejícími úkony, patří mezi ty více frustrující.

Nesmyslné datové typy, zkomolené časové údaje, nefungující transformace.

In [1]:
from pandas import DataFrame

In [2]:
uzivatele_df = DataFrame({'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
                          'Age': [None, 20, None, 22, 23],
                          'Height_cm': [160, 170, 175, None, 165],
                          'Weight_kg': [50, 65, 70, 80, 55]})

In [3]:
uzivatele_df

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70
3,David,22.0,NaN,80
4,Eve,23.0,165.0,55


Ihned po načtení je nejlepší **data prozkoumat**.

Nejenom datové typy ale také mít povědomí **o chybějících hodnotách**:

In [4]:
uzivatele_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       5 non-null      object 
 1   Age        3 non-null      float64
 2   Height_cm  4 non-null      float64
 3   Weight_kg  5 non-null      int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 288.0+ bytes


<br>

Pomocí metody `info` si uděláš aspoň povrchní přehled o hodnotách, datových typech a chybějících hodnotách.

<br>

### Kolik mi chybí údajů?

---

Vždycky přistupuj k datům s mírnou skepsí.

Otázkou potom není, *JESTLI* mi chybí data, ale *KOLIK* dat mi chybí **a kde**.

In [5]:
chybejici_hodnoty = uzivatele_df.isnull()  # isna()

In [6]:
chybejici_hodnoty

,Name,Age,Height_cm,Weight_kg
0,False,True,False,False
1,False,False,False,False
2,False,True,False,False
3,False,False,True,False
4,False,False,False,False


<br>

Pomocí metody `isnull` nahradíš všechny chybějící hodnoty v tabulce/sloupcích pomocí `True`.

Pokud nechybí, nahradí s `False`.

In [7]:
type(chybejici_hodnoty)

pandas.core.frame.DataFrame

Takový výstup ale není zcela reprezentativní.

<br>

Proto je potřeba, sečíst všechny `True` hodnoty **po sloupcích**:

In [8]:
soucet_chybejich_hodnot = uzivatele_df.isnull().sum()

In [9]:
soucet_chybejich_hodnot

Name         0
Age          2
Height_cm    1
Weight_kg    0
dtype: int64

<br>

Opět v jednotkách to nemusí zcela **prozrazovat chybovost**.

Nejlepším indikátorem ale bývají **procenta z celkového množství hodnot ve sloupci**:

In [10]:
uzivatele_df.shape[0]

5

<br>

... v tomto případě jde o počet záznamů/ řádků v tabulce.

In [11]:
celkem_zaznamu = uzivatele_df.shape[0]  # 0 ... Indexy, 1 ... sloupce

In [12]:
v_procentech = round(soucet_chybejich_hodnot / celkem_zaznamu * 100, 1)

In [13]:
v_procentech

Name          0.0
Age          40.0
Height_cm    20.0
Weight_kg     0.0
dtype: float64

<br>

Případně celkové množství záznamů ve všech sloupcích:

In [14]:
vsechny_bunky = uzivatele_df.shape[0] * uzivatele_df.shape[1]  # 5 * 4

In [15]:
vsechny_bunky

20

In [16]:
vsechny_chybejici_hodnoty = chybejici_hodnoty.sum()            # 3

In [17]:
vsechny_chybejici_hodnoty

Name         0
Age          2
Height_cm    1
Weight_kg    0
dtype: int64

In [18]:
soucet_chybejich_hodnot.sum()

3

In [19]:
celkem_v_procentech = round(soucet_chybejich_hodnot.sum() / vsechny_bunky * 100, 1)

In [20]:
celkem_v_procentech

15.0

<br>

V této ukázce tedy chybí 15 % dat a to není málo!

<br>

### Co s chybějícími hodnotami?

---

Je důležité vědět, proč máš takové množství chybějících hodnot.

Chybí ti data, protože neexistují, nebo protože nejsou součástí záznamů v datasetu.

V takových krocích je obvykle nutné, pochopit podstatu údajů (dokumentace, specifikace,..).

Nejjednodušší způsoby, jak s chybějícími daty naložit (ale ne efektivní!!):
1. Zahodit chybějící hodnoty,
2. doplnit chybějící hodnoty.

#### Zahodit chybějící hodnoty
---

In [21]:
# df_uzivatele.dropna?

In [22]:
uzivatele_df

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70
3,David,22.0,NaN,80
4,Eve,23.0,165.0,55


In [23]:
uzivatele_df.dropna()  # axis=0

,Name,Age,Height_cm,Weight_kg
1,Bob,20.0,170.0,65
4,Eve,23.0,165.0,55


<br>

V tento okamžik si zahodíš všechny záznamy, které **postrádaly nějakou hodnotu**.

Někdy je jednodušší zahodit sloupeček, který má chybějící hodnotu:

In [24]:
uzivatele_df.dropna(axis=1)

,Name,Weight_kg
0,Alice,50
1,Bob,65
2,Charlie,70
3,David,80
4,Eve,55


#### Doplnit automaticky hodnoty

---

In [25]:
# df_uzivatele.fillna?

In [26]:
uzivatele_df

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70
3,David,22.0,NaN,80
4,Eve,23.0,165.0,55


In [27]:
novy_df = uzivatele_df.fillna('Nula')  # replace: "Nan" --> 0

In [28]:
novy_df['Age'] * 2

0    NulaNula
1        40.0
2    NulaNula
3        44.0
4        46.0
Name: Age, dtype: object

In [29]:
uzivatele_df

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70
3,David,22.0,NaN,80
4,Eve,23.0,165.0,55


<br>

Pomocí funkce `fillna` můžeš vyplnit místo chybějících hodnot předdefinovanou vlastní hodnotu.

Nebo pomocí argumentu pro `method` pracovat s elegantnějším zadáním.

Nahradit chybějící hodnoty hodnotou, která následuje bezprostředně za ní ve stejném sloupci (To má velký smysl u souborů dat, kde mají pozorování nějaké logické pořadí.)

In [30]:
uzivatele_df.fillna(method='bfill', axis=0)

,Name,Age,Height_cm,Weight_kg
0,Alice,20.0,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,22.0,175.0,70
3,David,22.0,165.0,80
4,Eve,23.0,165.0,55


In [31]:
uzivatele_df

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70
3,David,22.0,NaN,80
4,Eve,23.0,165.0,55


In [32]:
uzivatele_df.fillna(method='ffill', axis=0)

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,20.0,175.0,70
3,David,22.0,175.0,80
4,Eve,23.0,165.0,55


<br>

Při volbě metody `ffill` může dělat problémy chybějící hodnota v prvním Indexu.

In [33]:
uzivatele_df.fillna(method='ffill', axis=0).fillna(19)

,Name,Age,Height_cm,Weight_kg
0,Alice,19.0,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,20.0,175.0,70
3,David,22.0,175.0,80
4,Eve,23.0,165.0,55


Pro doplnění **nedoplněných hodnot** ve spolupráci s metodou `fillna` můžeš tuto metoda spustit opakovaně.

In [34]:
import numpy

In [35]:
numpy.isnan(0)

False

Založím novou kopii objektu z původní tabulky: 

In [36]:
novi_uzivatele_df = uzivatele_df.copy()

<br>

Zapíšu existující hodnoty do nového sloupce `Age_NEW`:

In [37]:
novi_uzivatele_df['Age_NEW'] = novi_uzivatele_df['Age']

<br>

Nahradíme prázdné řádky s doplňujícím výpočtem:

In [38]:
chybejici = novi_uzivatele_df['Age'].isna()
novi_uzivatele_df.loc[chybejici, 'Age_NEW'] = novi_uzivatele_df.loc[chybejici, 'Height_cm'] * novi_uzivatele_df.loc[chybejici, 'Weight_kg']

In [39]:
novi_uzivatele_df.head()

,Name,Age,Height_cm,Weight_kg,Age_NEW
0,Alice,NaN,160.0,50,8000.0
1,Bob,20.0,170.0,65,20.0
2,Charlie,NaN,175.0,70,12250.0
3,David,22.0,NaN,80,22.0
4,Eve,23.0,165.0,55,23.0


<img src="https://imgs.search.brave.com/lUUqjUIBtDUFPXtoZsXuQckIeZDaKPxCsaKdK1kJuFw/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA5LzM4LzAxLzE1/LzM2MF9GXzkzODAx/MTU4MF9leVZ5Um9z/Z20wakNsVzhMRm1l/R0dpeDNJQ0tXdmhV/WS5qcGc" width="160" style="margin-left:auto; margin-right:auto"/>



### Encoding, znakové sady

---

Problémy s *kódováním* jsou běžné.

Jde o proces, který mapuje **bajtové stringy** (`0110101011`) na **uživatelsky čitelný text** (`"ahoj!"`).

Tento problém nastane, pokud se snažíš načíst zdroj v jiné kódovací sadě, než byl soubor zapsaný.

Protože je těchto sad hodně, občas skončíš s *escapovanými znaky*, nebo s neznámými kliky-háky:

In [40]:
print('æ–‡å')

æ–‡å


<br>

Případně pokud nelze domapovat znaky:

��

<br>

V **Pythonu 2** nebylo lehké *encoding* zajistit.

V **Pythonu 3** je celý proces o dost jednodušší.

Standardem pro práci bývá obyčejně UTF-8. Na to ale zrovna v našich končinách nemůžeš spoléhat.

In [41]:
veta = "Matouš zaplatil 100 $"

In [42]:
# veta?

<br>

Údaj, který vidíš výš je datový typ `str`.

In [43]:
prevedena_veta = veta.encode("utf-8", errors="replace")

In [44]:
prevedena_veta

b'Matou\xc5\xa1 zaplatil 100 $'

In [45]:
# prevedena_veta?

<br>

Dál je možné, převést `str` na `bytes`. Tedy sekvenci čísel.

Jak ale teď tato sekvence vypadá:

In [46]:
prevedena_veta

b'Matou\xc5\xa1 zaplatil 100 $'

<br>

Celou větu můžeš zpátky kódovat pomocí funkce `decode` z `bytes` na `str`:

In [47]:
print(prevedena_veta.decode("utf-8"))

Matouš zaplatil 100 $


<br>

Pokud ale zvolíš jinou sadu, nemusíš dostat stejné hodnoty:

In [48]:
print(prevedena_veta.decode("windows-1250"))

MatouĹˇ zaplatil 100 $


In [49]:
# print(prevedena_veta.decode("ascii"))

<br>

Toto a další neúspěšné postupy je samozřejmě špatně a ty se tomu chceš určitě vyhnout.

In [50]:
from pandas import read_csv

In [51]:
# chybna_sada = read_csv("../onsite/cviceni_3.csv")

<br>

Ne vždy je možné, defaultně pracovat se znakovou sadou **UTF-8**.

Ve výstupu je vidět, že jde o směs několika sad a ty musíš vyzkoušet, jaká bude fungovat.

<br>

Pomocí knihovny `sys` zkontroluješ, jakou znakovou sadu interně používáš:

In [52]:
import sys

In [53]:
sys.getdefaultencoding()

'utf-8'

<br>

Ještě potřebuješ nástroj, který umí ověřit znakovou lokálního souboru:

#### Řešení pomocí knihovny chardet

---

In [54]:
# !pip install chardet

In [55]:
from chardet.universaldetector import UniversalDetector

In [56]:
detector = UniversalDetector()

In [57]:
for line in open("../onsite/cviceni_3.csv", 'rb'):
    detector.feed(line)
    if detector.done:
        break

detector.close()
print(detector.result)

{'encoding': 'Windows-1252', 'confidence': 0.6943073341094295, 'language': ''}


<br>

#### Řešení pomocí charset-normalizer

---

In [58]:
# !pip install charset-normalizer

In [59]:
import charset_normalizer

In [60]:
vysledky = charset_normalizer.from_path("../onsite/cviceni_3.csv")

In [61]:
best = vysledky.best()

In [62]:
best.percent_coherence

52.54

In [63]:
best.encoding

'cp1250'

<br>

Pomocí stringu knihovně předám lokální soubor, který je řádek po řádku kontrolován:

<br>

Pomocí knihovny `chardet` / `charset_normalizer` identifikuješ, o jaké znakové sady jde:

Nyní máš lepší představu o tom, jakou znakovou sadu soubor používá.

Opatrně na délku `str`, který detekuješ.

Pokud je příliš krátký, může to ovlivnit výsledek.

Naopak pokud je příliš dlouhý, může trvat jeho načtení.

In [64]:
spravna_sada = read_csv("../onsite/cviceni_3.csv", encoding="Windows-1252")

In [65]:
spravna_sada.head()

,id;jmeno;prijmeni;vek;mesto;uzivatel;narozeni
0,1345;Jitka;Prokopová;46;praha;N;4/8/2021
1,1346;Jan;Hornych;18;Praha;N;21/4/1995
2,1347;Lumír;Navrátil;46;Olomouc;N;4/5/1987
3,1348;Pavel;Koutný;35;praha;Y;16/2/1995
4,1349;Ivan;Èížek;35;Liberec;N;5/5/2008


In [66]:
spravna_sada = read_csv("../onsite/cviceni_3.csv",
                        encoding="cp1250",
                        sep=';')

In [67]:
spravna_sada.head()

,id,jmeno,prijmeni,vek,mesto,uzivatel,narozeni
0,1345,Jitka,Prokopová,46,praha,N,4/8/2021
1,1346,Jan,Hornych,18,Praha,N,21/4/1995
2,1347,Lumír,Navrátil,46,Olomouc,N,4/5/1987
3,1348,Pavel,Koutný,35,praha,Y,16/2/1995
4,1349,Ivan,Čížek,35,Liberec,N,5/5/2008


In [68]:
spravna_sada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    int64 
 1   jmeno     100 non-null    object
 2   prijmeni  100 non-null    object
 3   vek       100 non-null    int64 
 4   mesto     100 non-null    object
 5   uzivatel  100 non-null    object
 6   narozeni  100 non-null    object
dtypes: int64(2), object(5)
memory usage: 5.6+ KB


<br>

Tentokrát soubor otevřeš pouze s varováním.

Pro příště je určitě výhodou vytvořit kopii takového souboru, kterou uložíš v ideálním kódování:

In [69]:
spravna_sada.to_csv("../onsite/cviceni_3_sada_utf8.csv", encoding="utf-8")

<img src="https://imgs.search.brave.com/EXc5_PuzQ1oh9QtKkq86VbHLTjcQKbX7sH3-di9Lrxc/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9jZG4y/Lmljb25maW5kZXIu/Y29tL2RhdGEvaWNv/bnMvdGhpbmdzLTEz/LzgwL21hdGNoaW5n/LWNvbXBhcmUtbWF0/Y2hpbmctY29sbGF0/ZS02NC5wbmc" width="160" style="margin-left:auto; margin-right:auto"/>



<br>

### Nekonzistentní data

---

Pokud ti data nechybí, ještě neznamená, že musí být nutně v pořádku:

In [70]:
from pandas import DataFrame

In [71]:
chybny_dataset = {
    "id": [111, 112, 113, 114, 115, 116, 117],
    "jmeno": ["Matous", "Marek", "Petr", "Filip", "Jan", "Lukas", "David"],
    "vek": [22, 29, 31, 55, 43, 61, 55],
    "zeme": ["Ceska republika", "Slovensko", "Nemecko", "Ceskarepublika", "Ceska Republika", "Rakousko", "Ceska Reapublika"]
}

In [72]:
zamestnanci_df = DataFrame(chybny_dataset)

In [73]:
zamestnanci_df

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceskarepublika
4,115,Jan,43,Ceska Republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska Reapublika


Různé zdroje, správci můžou způsobit nejednotný zápis a dopustit se *nekonzistence*.

In [74]:
zamestnanci_df["jmeno"].unique()

array(['Matous', 'Marek', 'Petr', 'Filip', 'Jan', 'Lukas', 'David'],
      dtype=object)

In [75]:
zamestnanci_df["zeme"].unique()

array(['Ceska republika', 'Slovensko', 'Nemecko', 'Ceskarepublika',
       'Ceska Republika', 'Rakousko', 'Ceska Reapublika'], dtype=object)

<br>

To můžou být jak malá velká písmena, tak různé znaky, chybějící mezery apod.

Odstranit je není náročné.

Náročné může být opět rozpoznání takové komplikace.

In [76]:
zamestnanci_df["zeme"].str.lower().unique()

array(['ceska republika', 'slovensko', 'nemecko', 'ceskarepublika',
       'rakousko', 'ceska reapublika'], dtype=object)

<br>

Některé chyby, ale můžou způsobit paseku. Třeba chybějící mezery.

Tady je nejlepší, pomoci si vhodným nástrojem.

<br>

Knihovny typu:
- `Rapidfuzz`,
- `difflib`,
- `SpaCy`,
- `Jellyfish`,
- `fuzzywuzzy`

#### Ukázka pomocí fuzzywuzzy

---

In [77]:
# !pip install fuzzywuzzy

In [78]:
import fuzzywuzzy
from fuzzywuzzy import process

/home/matous/projects/python-data/env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<br>

Pro menší datasety s chybějícími mezerami aj., může tato knihovna pracovat prakticky sama.

Účel této knihovny je rozpoznat podobné řetězce, jaké

In [79]:
shodne = fuzzywuzzy.process.extract("Ceska republika",  # 'ceska repoblika'
                                    zamestnanci_df["zeme"],
                                    limit=6,
                                    scorer=fuzzywuzzy.fuzz.token_sort_ratio)

<br>

Dostávám **dvourozměrné pole** (matici), která obsahuje jednotlivé výstupy.

Mezi výstupy řadíme:
1. Porovnávaný string,
2. pravděpodobnost, se kterou odpovídá zadanému stringu.

In [80]:
shodne

[('Ceska republika', 100, 0),
 ('Ceska Republika', 100, 4),
 ('Ceskarepublika', 97, 3),
 ('Ceska Reapublika', 97, 6),
 ('Slovensko', 25, 1),
 ('Nemecko', 18, 2)]

In [81]:
zamestnanci_df["zeme"].unique()

array(['Ceska republika', 'Slovensko', 'Nemecko', 'Ceskarepublika',
       'Ceska Republika', 'Rakousko', 'Ceska Reapublika'], dtype=object)

<br>

Nyní můžeš vidět shodné `str` se zvoleným zadáním, jak to vidí `fuzzywuzzy`.

Pro nahrazení je optimální nachystat vhodnou uživatelskou funkci:

In [82]:
nejblizsi_shoda = [shoda[0] for shoda in shodne if shoda[1] >= 90]

<br>

Izoluji jen ty shody, které se z zádáním shodují více než na 90%:

In [83]:
nejblizsi_shoda

['Ceska republika', 'Ceska Republika', 'Ceskarepublika', 'Ceska Reapublika']

In [84]:
shodujici_udaje = zamestnanci_df["zeme"].isin(nejblizsi_shoda)  # ~membership testing

In [85]:
shodujici_udaje

0     True
1    False
2    False
3     True
4     True
5    False
6     True
Name: zeme, dtype: bool

In [86]:
zamestnanci_df

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceskarepublika
4,115,Jan,43,Ceska Republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska Reapublika


In [87]:
zamestnanci_df.loc[1]  # Index

id             112
jmeno        Marek
vek             29
zeme     Slovensko
Name: 1, dtype: object

In [88]:
zamestnanci_df.loc[1, 'zeme']

'Slovensko'

In [89]:
zamestnanci_df.loc[[0, 3, 4, 6], "zeme"]

0     Ceska republika
3      Ceskarepublika
4     Ceska Republika
6    Ceska Reapublika
Name: zeme, dtype: object

In [90]:
zamestnanci_df.loc[shodujici_udaje, "zeme"]

0     Ceska republika
3      Ceskarepublika
4     Ceska Republika
6    Ceska Reapublika
Name: zeme, dtype: object

In [91]:
zamestnanci_df.loc[shodujici_udaje, "zeme"] = "Ceska republika"

In [92]:
zamestnanci_df

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceska republika
4,115,Jan,43,Ceska republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska republika


In [93]:
def nahrad_shody_stringem(dframe, jmeno_sloupce, vzor, min_shoda=90):
    vsechny_stringy = dframe[jmeno_sloupce].unique()
    
    shody = fuzzywuzzy.process.extract(vzor, vsechny_stringy, limit=5,
                                       scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    nejblizsi_shoda = [shoda [0] for shoda in shody if shoda[1] >= min_shoda]
    
    shodujici_zaznam = dframe[jmeno_sloupce].isin(nejblizsi_shoda)
    dframe.loc[shodujici_zaznam, jmeno_sloupce] = vzor

In [94]:
nahrad_shody_stringem(dframe=zamestnanci_df, jmeno_sloupce="zeme", vzor="Ceska republika")

#### Ukázka s RapidFuzz

---

In [95]:
# !pip install RapidFuzz

In [96]:
from rapidfuzz import fuzz, process

In [97]:
def nahrad_shody_stringem(dframe, jmeno_sloupce, vzor, min_shoda=90):
    vsechny_stringy = dframe[jmeno_sloupce].unique()

    shody = process.extract(
        vzor,
        vsechny_stringy,
        scorer=fuzz.token_sort_ratio,
        limit=5
    )

    nejblizsi_shody = [shoda[0] for shoda in shody if shoda[1] >= min_shoda]

    shodujici_zaznam = dframe[jmeno_sloupce].isin(nejblizsi_shody)
    dframe.loc[shodujici_zaznam, jmeno_sloupce] = vzor

    return dframe

In [98]:
nahrad_shody_stringem(zamestnanci_df, "zeme", "Ceska Republika", min_shoda=85)

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska Republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceska Republika
4,115,Jan,43,Ceska Republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska Republika


In [99]:
zamestnanci_df

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska Republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceska Republika
4,115,Jan,43,Ceska Republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska Republika


<br>

#### **🧠 CVIČENÍ 🧠, procvič si práci s chybným datasetem, 1**

In [100]:
import pandas as pd
from fuzzywuzzy import process

data_df = pd.DataFrame({
    "city": ["Praag", "Londn", "New York", "Tokio", "Sidney", "London", "Praha", "Nwe York", "Sydney", "Tky"]
})

correct_cities = ["Prague", "London", "New York", "Tokyo", "Sydney"]

<details>
    <summary>▶️ Řešení</summary>
    
```python
import pandas as pd

data = pd.DataFrame({
    "city": ["Praag", "Londn", "New York", "Tokio", "Sidney", "London", "Praha", "Nwe York", "Sydney", "Tky"]
})

correct_cities = ["Prague", "London", "New York", "Tokyo", "Sydney"]

def fix_city_name_all(city, correct_cities, threshold=65):
    matches = process.extract(city, correct_cities, limit=3)
    best_match = matches[0]
    return best_match[0] if best_match[1] > threshold else city

data["corrected_city"] = [
    fix_city_name_all(city, correct_cities) for city in data["city"]
]

print(data)
```
</details>

<br>

#### **🧠 CVIČENÍ 🧠, procvič si práci s chybným datasetem, 2**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Nahraj soubor do `DataFrame`, jméno souboru `cviceni_3.csv` (správný encoding, správné rozdělení),
2. projdi sloupce, zkontroluj, kde je problém s daty,
3. vypiš všechny záznamy, které mají ve sloupci `mesto` hodnotu `Praha`.

In [101]:
from pandas import read_csv

In [102]:
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process

<details>
    <summary>▶️ Řešení</summary>
    
```python
df_ukazka = read_csv("cviceni_3.csv", encoding="Windows-1250", delimiter=";")

def nahrad_shody_stringem(df, vzor, vyber, min_shoda=85):
    shody = fuzzywuzzy.process.extract(vzor, vyber, limit=20,
                                       scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    nejblizsi_shoda = [shoda[0] for shoda in shody if shoda[1] >= min_shoda]

    shodujici_zaznam = vyber.isin(nejblizsi_shoda)
    df.loc[shodujici_zaznam, "mesto"] = vzor
    
nahrad_shody_stringem(df_ukazka, "Praha", df_ukazka["mesto"])
vystup = df_ukazka[df_ukazka["mesto"] == "Praha"]


# Alternativní řešení
data = pd.read_csv(
    encoding = 'cp1250',
    sep = ';'
)

def nahrad_shody_stringem(dframe, sloupec, vzor, min_shoda=60):
    vsechny_stringy = dframe[sloupec].unique()
    shody = fuzzywuzzy.process.extract(vzor, vsechny_stringy, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    nejblizsi_shoda = [shoda [0] for shoda in shody if shoda[1] >= min_shoda]
    shodujici_zaznam = dframe[sloupec].isin(nejblizsi_shoda)
    dframe.loc[shodujici_zaznam, sloupec] = vzor

print( 'Problémy s daty:' )
display( data[~data['narozeni'].str.contains('\\d{1,2}/\\d{1,2}/\\d{4}', regex= True, na=False)] )
display( data['mesto'].unique( ) )

#oprava narození
data['narozeni'] = data['narozeni'].astype( 'datetime64' )
#oprava města
nahrad_shody_stringem(dframe=data, sloupec="mesto", vzor="Praha")
print( 'Výsledek:' )
data.query( "mesto == 'Praha'" )
```
</details>

---